In [1]:
%pip install transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 26.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.8 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.6.1
    Uninstalling fsspec-2024.6.1:
      Successfully uninstalled fsspec-2024.6.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.3.1+cu121 requ

In [2]:
import torch
from datasets import load_dataset

In [3]:
from huggingface_hub import notebook_login

notebook_login()

In [4]:
device = torch.device(
    "cuda" if torch.cuda.is_available() else "cpu")
print(f'Using device. {device}')

Using device. cuda


In [18]:
import pandas as pd
def prepare_data(csv_path):
    df = pd.read_csv(csv_path)
    return Dataset.from_pandas(df[['input', 'output']])

csv_path = '/content/drive/MyDrive/Colab Notebooks/first_500_games_fen (1).csv'
dataset = prepare_data(csv_path)

In [10]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, Trainer, TrainingArguments, DataCollatorForLanguageModeling

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")


In [20]:
from datasets import Dataset

In [23]:
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})

# Update the model
model = GPT2LMHeadModel.from_pretrained("gpt2")
model.resize_token_embeddings(len(tokenizer))

Embedding(50258, 768)

In [24]:
def tokenize_function(examples):
    input_encodings = tokenizer(examples['input'], padding="max_length", truncation=True, max_length=128)
    output_encodings = tokenizer(examples['output'], padding="max_length", truncation=True, max_length=128)

    return {
        'input_ids': input_encodings['input_ids'],
        'attention_mask': input_encodings['attention_mask'],
        'labels': output_encodings['input_ids']
    }

In [25]:
tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/47025 [00:00<?, ? examples/s]

In [26]:
training_args = TrainingArguments(
    output_dir="./result",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=True,
    save_total_limit = 3,
    load_best_model_at_end=False
)


data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets,
    eval_dataset=tokenized_datasets,
    data_collator=data_collator,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [27]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,1.068900,1.022926
2,0.975900,0.936394
3,0.947100,0.897101


TrainOutput(global_step=70539, training_loss=1.067804981253808, metrics={'train_runtime': 11518.7912, 'train_samples_per_second': 12.247, 'train_steps_per_second': 6.124, 'total_flos': 9215443353600000.0, 'train_loss': 1.067804981253808, 'epoch': 3.0})

In [28]:
trainer.push_to_hub()

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

events.out.tfevents.1723052894.1b9767a5f586.177.0:   0%|          | 0.00/36.4k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/498M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/nurke/result/commit/b3fb0bf1613b8b8338a20e7a870c7804bdeee5b6', commit_message='End of training', commit_description='', oid='b3fb0bf1613b8b8338a20e7a870c7804bdeee5b6', pr_url=None, pr_revision=None, pr_num=None)